# Arboles Binarios

Un método de búsqueda por ı́ndices son los KD-trees, los cuales se basan en árboles
binarios. El método para construir los árboles binarios realiza cortes sobre la mediana de un
eje y produce un árbol de altura $log(n)$. El árbol puede ser construido de manera recursiva
con un costo $O(n \operatorname{log} n)$, el cual efectivamente es menor que la búsqueda exhaustiva $O(n2)$.
La Figura 1 muestra una árbol binario. Los hijos de cada nodo corresponden a la posición
de la mediana del eje seleccionado para cada profundidad.



In [38]:
import networkx as nx
import pandas as pd
import numpy as np

from math import radians, cos, sin, asin, sqrt

class Node:pass

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def kdtree_create(point_list,depth=0):
    if len(point_list)==0:
        return
    D = len(point_list.columns) # assumes all points have the same dimension
    axis = depth % D
    key=point_list.columns[axis]
    A=point_list.sort_values(by=key)
    median = len(point_list)/2 # choose median
    # Create node and construct subtrees
    node = Node()
    node.location = A.iloc[median]
    node.left_child = kdtree_create(A.iloc[0:median], depth+1)
    node.right_child = kdtree_create(A.iloc[median+1:], depth+1)
    return node

def get_nearest_node(G,point):
    node_distances={}
    for g in G.nodes():
        dist=haversine(np.float(G.node[g]['y']),np.float(G.node[g]['x']),point[0],point[1])
        node_distances.update({g:dist})
    lookup_list = pd.Series(node_distances)
    min_dist= min(node_distances.values())
    return lookup_list[lookup_list.values == min_dist].index[0]

def get_point_list(G):
    point_list=pd.DataFrame()
    for g in G.nodes():
        lat=np.float(G.node[g]['y'])
        lon=np.float(G.node[g]['x'])
        df=pd.DataFrame([[lat,lon]],columns=['lat','lon'])
        if (len(point_list)==0):
            point_list=df
        else:
            point_list=point_list.append(df,ignore_index=True)
    return point_list

G=nx.read_graphml('data/talca_ciclovias.graphml')
print nx.info(G)
origin_point = (-35.434415,-71.620053)
destination_point = (-35.425901, -71.666645)

point_list=get_point_list(G)

kdtree=kdtree_create(point_list)


Name: Talca,Chile
Type: MultiDiGraph
Number of nodes: 7607
Number of edges: 20233
Average in degree:   2.6598
Average out degree:   2.6598


De manera de consultar por los vecinos más próximos (nn), descendemos por el árbol y
mantenemos un punto candidato a ser el nn y un valor máximo conocido de la distancia
al punto de consulta. Luego chequeamos los sub-árboles dependiendo del eje de corte
continuamos por una de las ramas. Un ejemplo tı́pico es la búsqueda por rangos en consultas
geoespaciales.

In [40]:
root_node=kdtree.location
print root_node
print kdtree.left_child.location
print kdtree.right_child.location

lat   -35.432749
lon   -71.681489
Name: 5641, dtype: float64
lat   -35.447087
lon   -71.662497
Name: 7147, dtype: float64
lat   -35.415658
lon   -71.644622
Name: 1918, dtype: float64


In [ ]:
def kdtree_knn(kdtree,point,n=1):
    #Retornar lista de n ciudades mas cercanas al punto point=(p1,p2)
    return None
def kdtree_range(kdtree,point,range=0):
    #Retornar lista de ciudades cercanas al punto point=(p1,p2) dentro de un rango
    return None